## 1. GPU確認

In [ ]:
!nvidia-smi

import torch
print(f"\nPyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Device: {torch.cuda.get_device_name(0)}")

## 2. Google Drive マウント（簡単！）

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

print("\n✅ Google Drive mounted at /content/drive")

## 3. パス設定

In [ ]:
import os

# ★Google Drive上のプロジェクトフォルダパスを設定★
GDRIVE_PROJECT_PATH = "/content/drive/MyDrive/atma22"

# データパス
GDRIVE_DATA_PATH = f"{GDRIVE_PROJECT_PATH}/data/raw/input"

# 出力パス（Drive上に直接保存）
GDRIVE_MODELS_PATH = f"{GDRIVE_PROJECT_PATH}/models"
GDRIVE_LOGS_PATH = f"{GDRIVE_PROJECT_PATH}/logs"
GDRIVE_SUBMISSION_PATH = f"{GDRIVE_PROJECT_PATH}/data/submission"
GDRIVE_FEATURES_PATH = f"{GDRIVE_PROJECT_PATH}/data/features"

# フォルダ作成
for path in [GDRIVE_MODELS_PATH, GDRIVE_LOGS_PATH, GDRIVE_SUBMISSION_PATH, GDRIVE_FEATURES_PATH]:
    os.makedirs(path, exist_ok=True)

print("=== Path Configuration ===")
print(f"Data: {GDRIVE_DATA_PATH}")
print(f"Models: {GDRIVE_MODELS_PATH}")
print(f"Logs: {GDRIVE_LOGS_PATH}")

# データ確認
!ls -lh {GDRIVE_DATA_PATH}/atmaCup22_metadata/

## 4. Gitクローン

In [ ]:
# ★GitHubリポジトリURLを設定★
GITHUB_REPO = "https://github.com/YOUR_USERNAME/atma_22_ca.git"
PROJECT_NAME = "atma_22_ca"

# Git clone
if not os.path.exists(f"/content/{PROJECT_NAME}"):
    !git clone {GITHUB_REPO} /content/{PROJECT_NAME}
    print(f"✅ Cloned {GITHUB_REPO}")
else:
    print(f"⚠️ {PROJECT_NAME} exists. Pulling latest...")
    !cd /content/{PROJECT_NAME} && git pull

%cd /content/{PROJECT_NAME}

## 5. パッケージインストール

In [ ]:
!pip install -q scikit-learn pandas numpy matplotlib seaborn tqdm

## 6. 設定ファイル更新（Drive上のパスを使用）

In [ ]:
# config.pyをDrive上のパスに書き換え
config_content = f'''
from pathlib import Path

# Google Drive上のパス
DIR_INPUT = Path("{GDRIVE_DATA_PATH}")
DIR_MODEL = Path("{GDRIVE_MODELS_PATH}")
DIR_LOG = Path("{GDRIVE_LOGS_PATH}")
DIR_SUBMISSIONS = Path("{GDRIVE_SUBMISSION_PATH}")
DIR_FEATURES = Path("{GDRIVE_FEATURES_PATH}")

# ローカルキャッシュは/content/に配置（高速化）
CACHE_DIR = Path("/content/cache")
CACHE_DIR.mkdir(exist_ok=True)
'''

with open('/content/atma_22_ca/configs/config.py', 'w') as f:
    f.write(config_content)

print("✅ Config updated for Google Drive mount")

## 7. モジュールインポート

In [ ]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

sys.path.append('/content/atma_22_ca')

from configs.config import *
from src.runner import Runner
from src.model_resnet_knn import ModelResNet50KNN
from src.util import Logger, Util, Metric

print(f"Input: {DIR_INPUT}")
print(f"Models: {DIR_MODEL}")

## 8. ロガー設定

In [ ]:
logger = Logger(path=DIR_LOG)

def get_run_name(model_type):
    return model_type + '_' + datetime.now().strftime("%Y%m%d%H%M")

logger.info("Logger initialized (Colab Mount)")

## 9. データ読み込み

In [ ]:
df_train = pd.read_csv(os.path.join(DIR_INPUT, 'atmaCup22_metadata', 'train_meta.csv'))
df_test = pd.read_csv(os.path.join(DIR_INPUT, 'atmaCup22_metadata', 'test_meta.csv'))

print(f"Train: {df_train.shape}")
print(f"Test: {df_test.shape}")

df_train['group'] = df_train['quarter'] + '_' + df_train['session'].astype(str)

print(f"\nGroups: {df_train['group'].nunique()}")
print(f"Labels: {df_train['label_id'].nunique()}")

## 10. パラメータ設定（GPU最適化）

In [ ]:
memo = "ResNet50 + KNN (Colab Mount)"
run_name = get_run_name("resnet50_knn_mount")

params = {
    'model_name': 'resnet50',
    'k': 5,
    'threshold': 0.5,
    'min2_threshold': 0.3,
    'batch_size': 64,
    'num_workers': 2,
    'use_cache': True,
}

print(f"Run: {run_name}")
print(f"Parameters: {params}")

## 11. CV設定

In [ ]:
cv_strategy = {
    'method': 'GroupKFold',
    'n_splits': 5,
    'shuffle': False,
    'random_state': 42
}

## 12. Runner作成

In [ ]:
runner = Runner(
    run_name=run_name,
    model_cls=ModelResNet50KNN,
    params=params,
    cv_strategy=cv_strategy,
    logger=logger
)

print(f"Runner created: {run_name}")

## 13. 5-Fold CV学習（GPU高速化）

In [ ]:
%%time
scores = runner.train_cv(
    df_train, 
    groups=df_train['group'].values
)

print("\n" + "="*80)
print(f"CV Score (Macro F1): {np.mean(scores):.5f} ± {np.std(scores):.5f}")
print("="*80)

## 14. テストデータ予測

In [ ]:
%%time
pred_test = runner.predict_cv(df_test)

print(f"Predictions shape: {pred_test.shape}")
print(f"\nDistribution:")
print(pred_test['label_id'].value_counts().sort_index())

## 15. 提出ファイル作成（Google Drive上に保存）

In [ ]:
submission_path = runner.save_submission(pred_test, run_name)

print(f"\n✅ Submission saved: {submission_path}")
print(f"\nFirst 10 predictions:")
print(pred_test.head(10))

# Google Drive上のパスも表示
print(f"\n📁 Google Drive path:")
print(f"  {GDRIVE_SUBMISSION_PATH}/{run_name}.csv")

## 16. kチューニング実験

In [ ]:
k_values = [3, 5, 7, 10, 15]
k_results = []

for k in k_values:
    predictions_list = []
    
    for i_fold in range(runner.n_splits):
        model = runner.load_model_cv(i_fold)
        
        if hasattr(model, 'test_similarities') and model.test_similarities is not None:
            pred = model.predict_with_custom_threshold(
                threshold=params['threshold'],
                min2_threshold=params['min2_threshold'],
                k=k
            )
            predictions_list.append(pred)
    
    if predictions_list:
        predictions_array = np.array(predictions_list)
        final_pred = []
        for i in range(predictions_array.shape[1]):
            values, counts = np.unique(predictions_array[:, i], return_counts=True)
            final_pred.append(values[np.argmax(counts)])
        
        unknown_count = np.sum(np.array(final_pred) == -1)
        unknown_ratio = unknown_count / len(final_pred) * 100
        
        k_results.append({
            'k': k,
            'unknown_count': unknown_count,
            'unknown_ratio': unknown_ratio
        })
        
        print(f"k={k:2d}: unknown={unknown_count:4d} ({unknown_ratio:5.2f}%)")

df_k_results = pd.DataFrame(k_results)
print("\n=== k-value tuning results ===")
print(df_k_results)

## 17. 結果確認

すべてのファイルはGoogle Drive上に保存されています：
- モデル: `/content/drive/MyDrive/atma22/models/`
- ログ: `/content/drive/MyDrive/atma22/logs/`
- 提出ファイル: `/content/drive/MyDrive/atma22/data/submission/`
- 特徴量キャッシュ: `/content/drive/MyDrive/atma22/data/features/`

Colabセッションを終了しても、これらのファイルはGoogle Driveに残ります。

In [ ]:
# 保存されたファイル一覧
print("=== Saved Models ===")
!ls -lh {GDRIVE_MODELS_PATH}/{run_name}/

print("\n=== Submission Files ===")
!ls -lh {GDRIVE_SUBMISSION_PATH}/

## まとめ

### ✅ 完了した作業
1. Google Driveマウント（1行）
2. GPU学習（5-fold CV）
3. テストデータ予測
4. kチューニング実験
5. 提出ファイル作成
6. **すべてGoogle Driveに自動保存**

### 📊 マウント方式のメリット
- ✅ セットアップが非常に簡単
- ✅ データコピー不要
- ✅ セッション切断後もデータ永続化

### ⚠️ 注意点
- I/O速度: rclone方式より遅い場合がある
- 大規模実験: 繰り返し実験にはrclone版推奨

### 🚀 次のステップ
- ローカルでGoogle Driveから結果を確認
- Prototype法との比較
- EfficientNet-B0で再実験